# Video Classification with Transformers

**Author:** [Sayak Paul](https://twitter.com/RisingSayak)<br>
**Date created:** 2021/06/08<br>
**Last modified:** 2023/22/07<br>
**Description:** Training a video classifier with hybrid transformers.

Ten przykład jest kontynuacją
[Klasyfikacja wideo według architektury CNN-RNN](https://keras.io/examples/vision/video_classification/)
przykład. Tym razem będziemy posługiwać się modelem opartym na Transformerze
([Vaswani i in.](https://arxiv.org/abs/1706.03762)), aby klasyfikować filmy. Możesz śledzić
[ten rozdział książki](https://livebook.manning.com/book/deep-learning-with-python-drugie-edition/chapter-11)
na wypadek gdybyś potrzebował wprowadzenia do Transformers (z kodem). Po przeczytaniu tego
na przykład będziesz wiedział, jak tworzyć hybrydowe modele oparte na transformatorach do zastosowań wideo
klasyfikacja działająca na mapach obiektowych CNN.

In [ ]:
!pip install -q git+https://github.com/tensorflow/docs

  Preparing metadata (setup.py) ... done


## Zbieranie danych

Podobnie jak w [poprzedniku](https://keras.io/examples/vision/video_classification/)
w tym przykładzie użyjemy podpróbkowanej wersji pliku
[Zbiór danych UCF101](https://www.crcv.ucf.edu/data/UCF101.php),
dobrze znany zbiór danych porównawczych. Jeśli chcesz operować na większej podpróbce lub
nawet cały zbiór danych, proszę zapoznać się z
[ten notatnik](https://colab.research.google.com/github/sayakpaul/Action-Recognition-in-TensorFlow/blob/main/Data_Preparation_UCF101.ipynb).

In [ ]:
!wget -q https://github.com/sayakpaul/Action-Recognition-in-TensorFlow/releases/download/v1.0.0/ucf101_top5.tar.gz
!tar -xf ucf101_top5.tar.gz

## Konfiguracja

In [ ]:
!pip install --upgrade keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 33.0 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.3.3 which is incompatible.


In [ ]:
import os
import keras
from keras import layers
from keras.applications.densenet import DenseNet121

from tensorflow_docs.vis import embed

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import imageio
import cv2

## Zdefiniuj hiperparametry

In [ ]:
MAX_SEQ_LENGTH = 20
NUM_FEATURES = 1024
IMG_SIZE = 128

EPOCHS = 5

## Przygotowywanie danych

W tym przykładzie będziemy głównie wykonywać te same kroki przygotowania danych, z wyjątkiem
następujące zmiany:

* Zmniejszamy rozmiar obrazu do 128x128 zamiast 224x224, aby przyspieszyć obliczenia.
* Zamiast używać wstępnie wyszkolonej sieci [InceptionV3](https://arxiv.org/abs/1512.00567),
używamy wstępnie przeszkolonego
[DenseNet121](http://openaccess.thecvf.com/content_cvpr_2017/papers/Huang_Densely_Connected_Convolutional_CVPR_2017_paper.pdf)
do ekstrakcji cech.
* Bezpośrednio dopasowujemy krótsze filmy do długości `MAX_SEQ_LENGTH`.

Najpierw załadujmy plik
[DataFrames](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).

In [ ]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")

center_crop_layer = layers.CenterCrop(IMG_SIZE, IMG_SIZE)


def crop_center(frame):
    cropped = center_crop_layer(frame[None, ...])
    cropped = keras.ops.convert_to_numpy(cropped)
    cropped = keras.ops.squeeze(cropped)
    return cropped

def load_video(path, max_frames=0, offload_to_cpu=False):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = frame[:, :, [2, 1, 0]]
            frame = crop_center(frame)
            if offload_to_cpu and keras.backend.backend() == "torch":
                frame = frame.to("cpu")
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    if offload_to_cpu and keras.backend.backend() == "torch":
        return np.array([frame.to("cpu").numpy() for frame in frames])
    return np.array(frames)


def build_feature_extractor():
    feature_extractor = DenseNet121(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.densenet.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["tag"]), mask_token=None
)
print(label_processor.get_vocabulary())


def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    labels = df["tag"].values
    labels = label_processor(labels[..., None]).numpy()

    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    for idx, path in enumerate(video_paths):

        frames = load_video(os.path.join(root_dir, path))

        if len(frames) < MAX_SEQ_LENGTH:
            diff = MAX_SEQ_LENGTH - len(frames)
            padding = np.zeros((diff, IMG_SIZE, IMG_SIZE, 3))
            frames = np.concatenate(frames, padding)

        frames = frames[None, ...]

        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                if np.mean(batch[j, :]) > 0.0:
                    temp_frame_features[i, j, :] = feature_extractor.predict(
                        batch[None, j, :]
                    )

                else:
                    temp_frame_features[i, j, :] = 0.0

        frame_features[idx,] = temp_frame_features.squeeze()

    return frame_features, labels

Total videos for training: 594
Total videos for testing: 224
29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
['CricketShot', 'PlayingCello', 'Punch', 'ShavingBeard', 'TennisSwing']


Wywołanie funkcji `prepare_all_videos()` na `train_df` i `test_df` zajmuje około 20 minut
kompletny. Z tego powodu, aby zaoszczędzić czas, tutaj pobieramy już wstępnie przetworzone tablice NumPy:

In [ ]:
!!wget -q https://git.io/JZmf4 -O top5_data_prepared.tar.gz
!!tar -xf top5_data_prepared.tar.gz

[]

In [ ]:
train_data, train_labels = np.load("train_data.npy"), np.load("train_labels.npy")
test_data, test_labels = np.load("test_data.npy"), np.load("test_labels.npy")

print(f"Frame features in train set: {train_data.shape}")

Frame features in train set: (594, 20, 1024)


## Budowa modelu opartego na transformatorze

Będziemy budować na podstawie udostępnionego kodu
[ten rozdział książki](https://livebook.manning.com/book/deep-learning-with-python-drugie-edition/chapter-11)
[Głębokie uczenie się z Pythonem (wyd. drugie)](https://www.manning.com/books/deep-learning-with-python)
przez François Cholleta.

Po pierwsze, są to warstwy samouważności, które tworzą podstawowe bloki Transformatora
niezależny od porządku. Ponieważ filmy są uporządkowanymi sekwencjami klatek, potrzebujemy naszego
Model transformatora uwzględniający informacje o zamówieniu.
Robimy to poprzez **kodowanie pozycyjne**.
Po prostu osadzamy pozycje klatek znajdujących się w filmach za pomocą pliku
[Warstwa „Osadzanie”](https://keras.io/api/layers/core_layers/embedding). Wtedy my
dodaj te osadzania pozycyjne do wstępnie obliczonych map obiektów CNN.

In [ ]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim
        )
        self.sequence_length = sequence_length
        self.output_dim = output_dim

    def build(self, input_shape):
        self.position_embeddings.build(input_shape)

    def call(self, inputs):
        inputs = keras.ops.cast(inputs, self.compute_dtype)
        length = keras.ops.shape(inputs)[1]
        positions = keras.ops.arange(start=0, stop=length, step=1)
        embedded_positions = self.position_embeddings(positions)
        return inputs + embedded_positions

Teraz możemy utworzyć warstwę podklasy dla Transformera.

## Funkcje użytkowe do treningu

## Trenowanie i wnioskowanie modelu

In [ ]:
trained_model = run_experiment()

Epoch 1/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5706 - loss: 2.1843
Epoch 1: val_loss improved from inf to 1.64706, saving model to /tmp/video_classifier.weights.h5
16/16 ━━━━━━━━━━━━━━━━━━━━ 37s 2s/step - accuracy: 0.5799 - loss: 2.1322 - val_accuracy: 0.4444 - val_loss: 1.6471
Epoch 2/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9541 - loss: 0.1710
Epoch 2: val_loss improved from 1.64706 to 0.34001, saving model to /tmp/video_classifier.weights.h5
16/16 ━━━━━━━━━━━━━━━━━━━━ 35s 2s/step - accuracy: 0.9546 - loss: 0.1686 - val_accuracy: 0.9000 - val_loss: 0.3400
Epoch 3/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9955 - loss: 0.0153
Epoch 3: val_loss did not improve from 0.34001
16/16 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.9954 - loss: 0.0152 - val_accuracy: 0.6222 - val_loss: 1.3465
Epoch 4/5
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9907 - loss: 0.0179
Epoch 4: val_loss improved from 0.34001 to 0.29268, saving model to /tmp/video_cla

**Uwaga**: Ten model ma ~4,23 miliona parametrów, czyli znacznie więcej niż sekwencja
model (99918 parametrów), którego użyliśmy w prequelu tego przykładu. Ten rodzaj
Model transformatora działa najlepiej w przypadku większego zbioru danych i dłuższego harmonogramu przedtreningowego.

In [ ]:
def prepare_single_video(frames):
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    if len(frames) < MAX_SEQ_LENGTH:
        diff = MAX_SEQ_LENGTH - len(frames)
        padding = np.zeros((diff, IMG_SIZE, IMG_SIZE, 3))
        frames = np.concatenate(frames, padding)

    frames = frames[None, ...]

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            if np.mean(batch[j, :]) > 0.0:
                frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
            else:
                frame_features[i, j, :] = 0.0

    return frame_features


def predict_action(path):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video(os.path.join("test", path), offload_to_cpu=True)
    frame_features = prepare_single_video(frames)
    probabilities = trained_model.predict(frame_features)[0]

    plot_x_axis, plot_y_axis = [], []

    for i in np.argsort(probabilities)[::-1]:
        plot_x_axis.append(class_vocab[i])
        plot_y_axis.append(probabilities[i])
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")

    plt.bar(plot_x_axis, plot_y_axis, label=plot_x_axis)
    plt.xlabel("class_label")
    plt.xlabel("Probability")
    plt.show()

    return frames

Wydajność naszego modelu jest daleka od optymalnej, ponieważ został wytrenowany na platformie
mały zbiór danych.